In [ ]:
%matplotlib inline
import pandas as pd
from io import StringIO, BytesIO
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image
from google.colab import files
import re

## some config
lw = 4.0              ## plot linewidth
figuresize = [20,12]  ## size of the plots

Input files have to contain the output of ib\_\*\_bw and ib\_\*\_lat with the "-a" option in the follwing form:

```
...
---------------------------------------------------------------------------------------
 local address: ...
 remote address: ...
---------------------------------------------------------------------------------------
 #bytes     #iterations    BW peak[Gb/sec]    BW average[Gb/sec]   MsgRate[Mpps]
 2          1000             0.09               0.087             5.444595
 4          1000             0.22               0.21   		   6.609449
 8          1000             0.43               0.42   		   6.651936
 16         1000             0.85               0.83   		   6.652351
 32         1000             1.70               1.62   		   6.648670
 64         1000             3.43               3.43   		   6.659067
 ...
 ```

In [ ]:
def read_perftest_output(filename):
    buffer=""
    with open(filename,'r') as f:
        filetext = f.read()

    # https://regex101.com/r/4NShAc/2
    regex = r"(?<=--\n)( #bytes.*)(?=\n--)"
    matches = re.search(regex, filetext, re.DOTALL)
    if matches:
        for line in matches.group(0).splitlines():
            if line.startswith(" #bytes"):
                buffer+= ",".join(line.replace("BW","").replace("[Gb/sec]","").\
                                    replace("[Mpps]","").replace("#","").\
                                    replace("[usec]","").replace("percentile","").split()) + "\n"
            else:
                buffer+= ",".join(line.split()) + "\n"
            
    return buffer

## Bandwidth

Upload the bandwidth file.

In [ ]:
upload = files.upload()
filename = list(upload.keys())[0] 

In [ ]:
df = pd.read_csv(StringIO(read_perftest_output(filename)))

In [ ]:
df.tail(5)

In [ ]:
plt.rcParams['figure.figsize'] = figuresize
plt.grid(True)
plt.xscale('log', basex=2)
plt.yscale('linear')
plt.ylim(0, df.average.max()+10)
plt.xlabel('Message Size [Bytes]')
plt.ylabel('Bandwidth [Gbit/s]')

plt.plot(df.bytes,df.average,linewidth=lw, label="Bandwidth")

plt.axhline(y=df.average.max(),color="green", linestyle='dashed')
plt.text(2,df.average.max()-3, 'max='+str(df.average.max())+'Gb/s', color='green')
plt.legend()  

## Latency

Upload the latency file.

In [ ]:
upload = files.upload()
filename = list(upload.keys())[0]

In [ ]:
df = pd.read_csv(StringIO(read_perftest_output(filename)))

In [ ]:
df.tail(5)

In [ ]:
plt.grid(True)
plt.xscale('log', basex=2)
plt.yscale('log', basey=2)
plt.xlabel('Message Size [Bytes]')
plt.ylabel('Latency [$\mu s$]')
plt.grid(True, which="minor", linestyle='--')


plt.plot(df.bytes, df.t_min, linewidth=lw, label="Latency")

plt.axhline(y=df.t_avg.min(),color="green", linestyle='dashed')
plt.text(df.bytes.max()/2,df.t_avg.min()+1, 'min='+str(df.t_avg.min())+'us', color='green')
plt.legend(loc="upper left")
plt.minorticks_on()